[Langchain full course : Chains](https://github.com/Coding-Crashkurse/Langchain-Full-Course/blob/main/chains.ipynb)

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, LLMChain

load_dotenv(find_dotenv())

False

In [2]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    model_kwargs=inference_modifier,
)

In [3]:
# from langchain_community.llms import Bedrock
# llm = Bedrock(model_id="anthropic.claude-v2")

In [4]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

# llm = OpenAI(temperature=0)
prompt_template = PromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="I ordered Pizza Salami and it was awesome!")

' {\n  "sentiment": "positive",\n  "subject": "pizza"\n}'

## Sequentials Chains
Sometimes you want to pass the output from one model to a another model. This can be done with different SequentialsChains

In [5]:
response_template = """
You are a helpful bot that creates a 'thank you' reponse text. 
If customers are unsatisfied, offer them a real world assitant to talk to. 
You will get a sentiment and subject as into and evaluate. 

text: {input}
"""
review_template = PromptTemplate(input_variables=["input"], template=response_template)
review_chain = LLMChain(llm=llm, prompt=review_template)

In [6]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, review_chain], verbose=True)

overall_chain.run(input="I ordered Pizza Salami and was aweful!")

/Users/tanaycho/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
 {
  "sentiment": "negative",
  "subject": "pizza"
}
 Thank you for your feedback. I'm sorry to hear you had a negative experience with the pizza. If you would like to speak with a human assistant to help resolve any issues, please let me know and I'd be happy to connect you. In the meantime, we appreciate you taking the time to share your thoughts so we can improve.

> Finished chain.


" Thank you for your feedback. I'm sorry to hear you had a negative experience with the pizza. If you would like to speak with a human assistant to help resolve any issues, please let me know and I'd be happy to connect you. In the meantime, we appreciate you taking the time to share your thoughts so we can improve."

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [7]:
from langchain.chains import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {review}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to german: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="german_translation"
)

overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "german_translation"],
)

overall_chain({"dish_name": "Pizza Salami", "experience": "It was awful!"})

/Users/tanaycho/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': " Here is a draft review of the Pizza Salami based on your experience:\n\nI was very disappointed with the pizza salami I ordered recently. The pizza dough was dry and crunchy, as if it had been overbaked. The sauce tasted bland and flavorless, like it came straight from a can with no spices added. The salami slices were rubbery and lacked any char or smoky flavor that good cured meats should have. Overall the pizza tasted very low quality, like it was made with cheap ingredients. \n\nI would not recommend getting the pizza salami from this restaurant. For the price I paid, I expected much better quality and flavor. The dough, sauce, and toppings were all subpar. I've had better tasting frozen pizzas! A good pizza salami should have a chewy, airy crust, zesty tomato sauce, and thin slices of peppery salami with a nice char. This pizza simply fell flat in all those departments. Unless the restaurant can seriously u

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [8]:
from langchain.llms import OpenAI
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [10]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]



destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=Bedrock(client=<botocore.client.BedrockRuntime object at 0x137c35490>, region_name='us-east-1', model_id='anthropic.claude-v2')),
 'neutral': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, not favoring any positive or negative aspects. Here is the text:\n{input}'), llm=Bedrock(client=<botocore.client.BedrockRuntime object at 0x137c35490>, region_name='us-east-1', model_id='anthropic.claude-v2')),
 'negative': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that is designed to find the negative aspects in a text. You analyze a text and show the potential downsides. Here is the text:\n{i

In [11]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.run("I ordered Pizza Salami for 9.99$ and it was awesome!")



> Entering new MultiPromptChain chain...


/Users/tanaycho/anaconda3/envs/genai/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


positive: {'input': 'I ordered Pizza Salami for 9.99$ and it was awesome!'}
> Finished chain.


' Here is an analysis that focuses on the positive aspects:\n\nI ordered Pizza Salami for 9.99$ and it was awesome!\n\nThis is a very positive review of a pizza order. The person ordered a Pizza Salami, which presumably refers to a salami pizza, for a reasonable price of $9.99. The key positive phrase is "it was awesome!" This indicates the customer was extremely satisfied and happy with the pizza they ordered. The exclamation point further emphasizes their enthusiasm. Overall, this is a great example of a positive customer experience with tasty, affordable pizza that exceeded expectations. By highlighting expressions of satisfaction and enjoyment, I aimed to focus on the upbeat, feel-good aspects of this text.'